# Flotorch ADK - UI-Configured Agent Example

This notebook returns an agent configured from the Flotorch UI.
Currently using hardcoded configuration in FlotorchADKAgent.

## Usage
1. Run mcp_server.py (provides 3 tools: Weather NYC, Time NYC, Capitals)
2. Run this notebook

## Alternative
Use custom config instead of hardcoded configuration.

## Requirements
- FLOTORCH_API_KEY and FLOTORCH_BASE_URL environment variables
- MCP server running for tools


# Setup and imports
import asyncio
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Add project path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..', '..'))

print("✓ Environment setup complete")


In [2]:
# Flotorch imports
from flotorch.adk.agent import FlotorchADKAgent
from google.adk.sessions import InMemorySessionService
from google.adk import Runner
from google.genai import types

print("✓ Flotorch ADK imports successful")


✓ Flotorch ADK imports successful


In [3]:
# Configuration
APP_NAME = "flotorch_ui_agent_example"
USER_ID = "flotorch_user_123"

print(f"App Name: {APP_NAME}")
print(f"User ID: {USER_ID}")


App Name: flotorch_ui_agent_example
User ID: flotorch_user_123


## Custom Configuration (Optional)

You can provide custom agent configuration instead of using hardcoded settings.


In [4]:
# Optional: Custom config (alternative to hardcoded configuration)
config = {
    "name": "sample_agent",
    "description": "Example Flotorch agent with user friendly assistance",
    "systemPrompt": "you are a helpful assistant",
    "llm": {
        "callableName": "openai/gpt-4o-mini"
    }
}

print("✓ Custom config defined")


✓ Custom config defined


In [12]:
# Access agent configured from Flotorch UI
# Currently hardcoded in FlotorchADKAgent
# Requires MCP server for tools (Weather NYC, Time NYC, Capitals)
flotorch_adk_client = FlotorchADKAgent(
    agent_name="flotorch_assistant",
    enable_memory=False,
    # config=config  # Use custom config or remove for hardcoded
)

print("✓ FlotorchADKAgent client created")


✓ FlotorchADKAgent client created


In [13]:
# Get the configured agent from UI
agent = flotorch_adk_client.get_agent()

print("✓ Agent retrieved from UI configuration")


✓ Agent retrieved from UI configuration


In [14]:
# Setup runner
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("✓ Runner setup complete")


✓ Runner setup complete


In [15]:
# Helper function to send messages
def run_single_turn(query, session_id, user_id):
    """Send message to Flotorch UI agent."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(
        user_id=user_id, 
        session_id=session_id, 
        new_message=content
    )

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that."

print("✓ Helper function defined")


✓ Helper function defined


In [16]:
# Create session and test UI agent
async def test_ui_agent():
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"Flotorch UI agent ready! Session: {session.id}")
    print("Tools available: Weather NYC, Time NYC, Capitals")
    
    # Test with a simple message
    test_message = "Hello! Can you help me?"
    response = run_single_turn(test_message, session.id, USER_ID)
    
    print(f"\nUser: {test_message}")
    print(f"Flotorch AI: {response}")
    
    return session

# Run the test
session = await test_ui_agent()


Flotorch UI agent ready! Session: 3788e97c-4786-46e8-933d-1cebd8a3d428
Tools available: Weather NYC, Time NYC, Capitals

User: Hello! Can you help me?
Flotorch AI: Of course! How can I assist you today?


In [17]:
# Interactive chat function with UI agent
def chat_with_ui_agent(message):
    """Simple function to chat with the UI-configured agent."""
    response = run_single_turn(message, session.id, USER_ID)
    print(f"You: {message}")
    print(f"Flotorch AI: {response}")
    return response

# Try the MCP tools (if MCP server is running)
print("=== Testing MCP Tools (requires mcp_server.py) ===")
print("Note: Make sure MCP server is running for these to work")

print("\n--- Weather Tool ---")
chat_with_ui_agent("What's the weather in New York?")

print("\n--- Time Tool ---")
chat_with_ui_agent("What time is it in New York?")

print("\n--- Capitals Tool ---")
chat_with_ui_agent("What's the capital of France?")


=== Testing MCP Tools (requires mcp_server.py) ===
Note: Make sure MCP server is running for these to work

--- Weather Tool ---
You: What's the weather in New York?
Flotorch AI: The weather in New York is sunny with a temperature of 30 degrees Celsius.

--- Time Tool ---
You: What time is it in New York?
Flotorch AI: The current time in New York is 2025-07-28 08:16:13 EDT-0400.

--- Capitals Tool ---
You: What's the capital of France?
Flotorch AI: The capital of France is Paris.


'The capital of France is Paris.'

## Summary

This notebook demonstrated:
- ✓ Using FlotorchADKAgent with UI configuration
- ✓ Custom config option as alternative to hardcoded settings
- ✓ Accessing pre-configured agents from Flotorch UI
- ✓ Testing MCP tools integration (Weather NYC, Time NYC, Capitals)

**Note**: For MCP tools to work, make sure to run `mcp_server.py` first!

You can now use `chat_with_ui_agent("your message")` to interact with the UI-configured agent!
